## Imports

In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.callbacks as callbacks
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
import random as r
import cv2
import os

## Constantes et variables globales

In [2]:
PATH_DATASET_FOLDER = 'Dataset_cesi/Dataset/'
RATIO_TRAIN = 0.8
RATIO_VAL = 0.15
RATIO_TEST = 0.05
USE_MULTIPROCESSING = False
WORKERS = 1 # si > 1, utilise threads si USE_MULTIPROCESSING = False, sinon utilise processus
BATCH_SIZE = 4
PATIENCE = 5
EPOCHS = 100
WIDTH = 640
HEIGHT = 640
CHANNELS = 3
PROB_GAUSSIAN = 0.6
PROB_PEPPER_SALT = 0.6
PROB_OBJECTIF_DEFAULT = 0.5
MODEL_NAME = 'livrable2.keras'

indices = []

## Dataset

### Exemple de bruit
| Type de Bruit          | Description                                                                                                                                                 |
|------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Bruit Gaussien**     | Ajoute une variation aléatoire des intensités de pixels selon une distribution normale (gaussienne). Il est couramment observé dans les images numériques dues aux capteurs d'image et aux conditions d'éclairage. |
| **Bruit Poivre et Sel**| Simule des pixels morts ou défectueux. Les pixels affectés deviennent blancs (sel) ou noirs (poivre). Commun dans les vieux films ou les capteurs défectueux. |
| **Bruit de Défault d'objectif**   | Représente des défauts de capture ou de transmission, tels que les rayures sur une photo ou des erreurs de scan. Il ajoute des lignes ou des artéfacts visibles comme du flou local à travers l'image.                                    |
| **Bruit Quantique**    | Provoqué par la fluctuation du nombre de photons détectés, typiquement dans des conditions de faible luminosité. Ce bruit est souvent modélisé comme un bruit de Poisson.                                               |
| **Bruit de Speckle**   | Souvent présent dans les applications utilisant la lumière cohérente, comme les radars ou les ultrasons. Ce bruit peut être simulé en multipliant l'image par un bruit gaussien.                                        |

Pour la suite de ce notebook, nous utiliserons les bruits Gaussien, Poivre et Sel, et de Défaut d'objectif.


In [ ]:
def add_noise_gaussian(image, mean=0, sigma=25):
    gauss = np.random.normal(mean, sigma, image.shape).reshape(image.shape)
    noisy_image = image + gauss
    noisy_image = np.clip(noisy_image, 0, 255)
    return noisy_image.astype(np.uint8)

def add_noise_salt_pepper(image, salt_prob=0.008, pepper_prob=0.008):
    num_salt = np.ceil(salt_prob * image.size)
    num_pepper = np.ceil(pepper_prob * image.size)

    # Ajout de sel
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    image[coords[0], coords[1]] = 255

    # Ajout de poivre
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
    image[coords[0], coords[1]] = 0

    return image.astype(np.uint8)

def add_noise_speckle(image, mean=0, sigma=0.1):
    gauss = np.random.normal(mean, sigma, image.shape).reshape(image.shape)
    noisy_image = image + image * gauss
    noisy_image = np.clip(noisy_image, 0, 255)
    return noisy_image.astype(np.uint8)

def add_noice_objectif(image):
    def add_noise_stripes(image, num_stripes_mean=2, num_stripes_std=1, color=(0, 0, 0)):
        rows, cols, _ = image.shape
        num_stripes = int(np.random.normal(num_stripes_mean, num_stripes_std))
        for _ in range(num_stripes):
            start = np.random.randint(0, cols - 1)
            end = start + np.random.randint(1, 5)  # Largeur de la rayure
            image[:, start:end] = color  # Rayure verticale noire
        return image
    
    def add_local_blur(image, num_areas_mean=1, num_areas_std=1, area_size_mean=180, area_size_std=20, sigma_mean=10, sigma_std=2):
        rows, cols, _ = image.shape
        num_areas = int(np.random.normal(num_areas_mean, num_areas_std))
        area_size = int(np.random.normal(area_size_mean, area_size_std))
        sigma = np.random.normal(sigma_mean, sigma_std)
        for _ in range(num_areas):
            center_x = np.random.randint(0, cols - area_size)
            center_y = np.random.randint(0, rows - area_size)
            sub_img = image[center_y:center_y+area_size, center_x:center_x+area_size]
            sub_img_blurred = cv2.GaussianBlur(sub_img, (7, 7), sigmaX=sigma, sigmaY=sigma)
            image[center_y:center_y+area_size, center_x:center_x+area_size] = sub_img_blurred
        return image
    
    def add_artifacts(image, num_artifacts_mean = 3, num_artifacts_std = 2, artifact_size_mean = 8, artifact_size_std = 3):
        rows, cols, _ = image.shape
        num_artifacts = int(np.random.normal(num_artifacts_mean, num_artifacts_std))
        artifact_size = int(np.random.normal(artifact_size_mean, artifact_size_std))
        for _ in range(num_artifacts):
            center_x = np.random.randint(0, cols - artifact_size)
            center_y = np.random.randint(0, rows - artifact_size)
            color = np.random.randint(0, 256, (3,))
            image[center_y:center_y+artifact_size, center_x:center_x+artifact_size] = color
        return image
    
    # Ajout de rayures
    image_stripes = add_noise_stripes(image)
    # Ajout de flou local
    image_blur = add_local_blur(image_stripes)
    # Ajout d'artefacts
    image_artifacts = add_artifacts(image_blur)
    return image_artifacts

# Charger l'image
image = Image.open("exemple.jpg")
image = np.array(image)

# Appliquer les différents bruits
noisy_gaussian = add_noise_gaussian(image.copy())
noisy_salt_pepper = add_noise_salt_pepper(image.copy())
noisy_speckle = add_noise_speckle(image.copy())
noisy_objectif = add_noice_objectif(image.copy())

# Afficher les résultats
fig, axs = plt.subplots(1, 4, figsize=(25, 5))
axs[0].imshow(noisy_gaussian, cmap='gray')
axs[0].set_title('Bruit Gaussien')
axs[0].axis('off')

axs[1].imshow(noisy_salt_pepper, cmap='gray')
axs[1].set_title('Bruit Poivre et Sel')
axs[1].axis('off')

axs[2].imshow(noisy_speckle, cmap='gray')
axs[2].set_title('Bruit de Speckle')
axs[2].axis('off')

axs[3].imshow(noisy_objectif, cmap='gray')
axs[3].set_title('Bruit de Défaut d\'objectif')
axs[3].axis('off')

plt.show()


### Métrics du dataset
On prendra comme dimension d'image pour notre modèle 640x640 pixels.

In [ ]:
def analyze_image_sizes(folder_path):
    heights = []
    widths = []
    
    # Parcourir le dossier et lire chaque image
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            with Image.open(img_path) as img:
                width, height = img.size
                widths.append(width)
                heights.append(height)
    
    # Afficher les distributions des largeurs et hauteurs
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(widths, bins=20, color='blue', alpha=0.7)
    plt.title('Distribution des largeurs')
    plt.xlabel('Largeur')
    plt.ylabel('Nombre d\'images')

    plt.subplot(1, 2, 2)
    plt.hist(heights, bins=20, color='green', alpha=0.7)
    plt.title('Distribution des hauteurs')
    plt.xlabel('Hauteur')
    
    plt.tight_layout()
    plt.show()

analyze_image_sizes(PATH_DATASET_FOLDER)

### Chargement des données

In [ ]:
class DatasetGenerator(Sequence):
    def _getshuffle(self, lenght, start, stop):
        global indices
        if len(indices) == 0 :
            # On initialise les indices
            indices = np.arange(lenght)
            np.random.shuffle(indices)
        return np.array(indices[start:stop])
    
    def __init__(self, ensemble, **kwargs):
        super().__init__(**kwargs)
        # Récupère le chemin de toutes les images d'un dossier
        def find_paths(folder_path):
            paths = []
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(folder_path, filename)
                    paths.append(img_path)
            return paths
        
        self.y_path = find_paths(PATH_DATASET_FOLDER)
        
        # Créer un ensemble de données
        if ensemble == 'train':
            start = 0
            stop = int(RATIO_TRAIN*len(self.y_path))
        elif ensemble == 'val':
            start = int(RATIO_TRAIN*len(self.y_path))
            stop = int((RATIO_TRAIN+RATIO_VAL)*len(self.y_path))
        elif ensemble == 'test':
            start = int((RATIO_TRAIN+RATIO_VAL)*len(self.y_path))
            stop = len(self.y_path)
        # Shuffle des données via les indices
        len_dataset = len(self.y_path)
        self.indices = self._getshuffle(len_dataset, start, stop)
        
        # Affichage des informations
        print(f"Création du générateur pour l'ensemble {ensemble}")
        print(f"Nombre d'images : {len(self.y_path)} / {len_dataset}")

    def __len__(self):
        return int(np.ceil(len(self.indices) / BATCH_SIZE))
    
    def __getitem__(self, index):
        start_index = index * BATCH_SIZE
        stop_index = (index + 1) * BATCH_SIZE
        chosen_indices = self.indices[start_index:stop_index]
        x,y = [],[]
        
        # On récupère les images et les labels
        for indices in chosen_indices:
            # Charger l'image
            img = Image.open(self.y_path[indices])
            img = img.resize((WIDTH, HEIGHT))
            img = np.array(img)
            img_noisy = img.copy()
            # Ajouter du bruit aléatoirement
            roll_gaussian = r.random()
            roll_salt_pepper = r.random()
            roll_objectif = r.random()
            if roll_gaussian < PROB_GAUSSIAN:
                img_noisy = add_noise_gaussian(img_noisy)
            if roll_salt_pepper < PROB_PEPPER_SALT:
                img_noisy = add_noise_salt_pepper(img_noisy)
            if roll_objectif < PROB_OBJECTIF_DEFAULT:
                img_noisy = add_noice_objectif(img_noisy)
            x.append(img_noisy)
            y.append(img)
        x = np.array(x)
        y = np.array(y) / 255.0 
        return x, y
 
    def on_epoch_end(self):
        # Shuffle des indices
        np.random.shuffle(self.indices)

train_generator = DatasetGenerator('train', use_multiprocessing=USE_MULTIPROCESSING, workers=WORKERS)
print('---------------------------------')
val_generator = DatasetGenerator('val', use_multiprocessing=USE_MULTIPROCESSING, workers=WORKERS)
print('---------------------------------')
test_generator = DatasetGenerator('test', use_multiprocessing=USE_MULTIPROCESSING, workers=WORKERS)

### Visualisation des données

In [ ]:
# Choissisez un générateur
generator = train_generator

r_index = r.randint(0, len(generator) - 1)
x, y = generator.__getitem__(r_index)
print(f'x.shape={x.shape} y.shape={y.shape}')

# Plot the results
fig, axs = plt.subplots(2, BATCH_SIZE, figsize=(20, 5))
for i in range(BATCH_SIZE):
    axs[0, i].imshow(y[i], cmap='gray')
    axs[0, i].axis('off')
    axs[1, i].imshow(x[i], cmap='gray')
    axs[1, i].axis('off')
plt.show()


## Modèle

### Chargement d'un modèle pré-existant

In [ ]:
model = load_model(MODEL_NAME)
model.summary()

### Création d'un modèle

In [ ]:
def auto_encoder():
    inputs = layers.Input(shape=(WIDTH, HEIGHT, CHANNELS))
    x = layers.Rescaling(1.0 / 255)(inputs)
    
    # Encoder
    x = layers.Conv2D(32, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(64, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(128, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    # Decoder
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(32, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2DTranspose(3, 3, strides=2, padding='same')(x)
    x = layers.Activation('sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=x, name='auto_encoder')
    model.compile(optimizer='adam', loss='mse')
    return model

def unet_auto_encoder():
    def conv_block(x, filters, strides=1):
        skip = layers.Conv2D(filters, 1, padding='same', strides=strides)(x)
        x = layers.Conv2D(filters, 3, padding='same', strides=1)(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU()(x)
        x = layers.Conv2D(filters, 3, padding='same', strides=strides)(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU()(x)
        x = layers.Concatenate()([x, skip])
        return x
    
    def transpose_conv_block(x, filters, strides=1):
        skip = layers.Conv2DTranspose(filters, 1, padding='same', strides=strides)(x)
        x = layers.Conv2DTranspose(filters, 3, padding='same', strides=1)(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU()(x)
        x = layers.Conv2DTranspose(filters, 3, padding='same', strides=strides)(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU()(x)
        x = layers.Concatenate()([x, skip])
        return x
        
    
    input = layers.Input(shape=(WIDTH, HEIGHT, CHANNELS))
    input = layers.Rescaling(1.0 / 255)(input)
    
    c1 = conv_block(input, 16, 2)
    c2 = conv_block(c1, 32, 2)
    c3 = conv_block(c2, 64, 2)
    
    p2 = transpose_conv_block(c3, 64, 2)
    p2 = layers.Concatenate()([p2, c2])
    p1 = transpose_conv_block(p2, 32, 2)
    p1 = layers.Concatenate()([p1, c1])
    p0 = transpose_conv_block(p1, 16, 2)
    p0 = layers.Concatenate()([p0, input])
    
    output = layers.Conv2D(3, 1, padding='same', activation='sigmoid')(p0)
    
    model = Model(inputs=input, outputs=output, name='unet_auto_encoder')
    model.compile(optimizer='adam', loss='mse')
    return model

model = unet_auto_encoder()
model.summary()

### Plot du modèle

In [ ]:
plot_model(model, to_file=f'Livrable2_{model.name}.jpg', show_shapes=True, show_layer_names=False, rankdir='TB')

### Entrainement du modèle

In [ ]:
# Callback d'early stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    verbose=1,
    restore_best_weights=True
)

# Boucle d'entraînement
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping],
)

# Plotting the training and validation loss
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

### Sauvegarde du modèle

In [10]:
model.save(MODEL_NAME)

## Evaluation du modèle

### Test unitaire

In [ ]:
# Choissisez un générateur
generator = train_generator

# Choix d'une image aléatoire
r_index = r.randint(0, len(generator) - 1)
x, y = generator.__getitem__(r_index)
r_index = r.randint(0, len(x) - 1)
x_sample = x[r_index]
y_sample = y[r_index]

# Prédiction
y_pred = model.predict(np.expand_dims(x_sample, axis=0))[0]

# Plot the results
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(y_sample)
axs[0].set_title('Image originale')
axs[0].axis('off')
axs[1].imshow(x_sample)
axs[1].set_title('Image bruitée')
axs[1].axis('off')
axs[2].imshow(y_pred)
axs[2].set_title('Image restaurée')
axs[2].axis('off')
plt.show()